# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [2]:
import graphlab

A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] GraphLab Create v1.8.3 started. Logging: /tmp/graphlab_server_1465874299.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [5]:
print sales.column_names(), len(sales.column_names())

['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt', 'sqft_lot_sqrt', 'bedrooms_square', 'floors_square'] 25


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [6]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Find what features had non-zero weight.

In [8]:
coef_all = model_all.get('coefficients')
coef_all.print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

In [9]:
coef_all['name'][coef_all['value'] != 0]

dtype: str
Rows: ?
['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ]

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [11]:
import numpy as np
l1_penalties = np.logspace(1, 7, num=13)
models = [graphlab.linear_regression.create(training, target='price', features=all_features, l2_penalty=0., 
                                          l1_penalty=l1_penalty, validation_set=None, verbose=False) 
          for l1_penalty in l1_penalties] 

In [12]:
RSSes = [sum((model.predict(validation) - validation['price'])**2) for model in models]

In [13]:
RSSes

[625766285142461.4,
 625766285362395.4,
 625766286057886.9,
 625766288257224.9,
 625766295212185.9,
 625766317206077.8,
 625766386760661.5,
 625766606749281.4,
 625767302791633.2,
 625769507643885.0,
 625776517727025.8,
 625799062845466.9,
 625883719085424.5]

In [14]:
best = sorted(zip(l1_penalties, models, RSSes), key=lambda x: x[2])[0]

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [15]:
print best[0], best[2]
print sum((best[1].predict(testing) - testing['price'])**2)

10.0 6.25766285142e+14
1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

        ANS: 18

In [16]:
best_coefs = best[1].get('coefficients')

In [17]:
best_coefs_NO0 = best_coefs[best_coefs['value'] != 0]
best_coefs_NO0

name,index,value,stderr
(intercept),None,18993.4272128,None
bedrooms,None,7936.96767903,None
bedrooms_square,None,936.993368193,None
bathrooms,None,25409.5889341,None
sqft_living,None,39.1151363797,None
sqft_living_sqrt,None,1124.65021281,None
sqft_lot,None,0.00348361822299,None
sqft_lot_sqrt,None,148.258391011,None
floors,None,21204.335467,None
floors_square,None,12915.5243361,None


In [18]:
print len(best_coefs_NO0)

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [19]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [20]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [21]:
models_2 = [graphlab.linear_regression.create(training, target='price', features=all_features, l2_penalty=0., 
                                          l1_penalty=l1_penalty, validation_set=None, verbose=False) 
          for l1_penalty in l1_penalty_values] 
NNZs = [model['coefficients']['value'].nnz() for model in models_2]

In [22]:
NNZs = np.array(NNZs)
NNZs

array([18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10,  6,  5,
        3,  1,  1])

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [23]:
print l1_penalty_values
l1_penalty_min = l1_penalty_values[NNZs > max_nonzeros][-1] # or .max()
l1_penalty_max = l1_penalty_values[NNZs < max_nonzeros][0] # or .min() 
print 'l1_penalty_min: ', l1_penalty_min
print 'l1_penalty_max: ', l1_penalty_max

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]
l1_penalty_min:  2976351441.63
l1_penalty_max:  3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

        ANS: l1_penalty_min:  2976351441.63, l1_penalty_max:  3792690190.73


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [24]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max, 20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [25]:
models_3 = [graphlab.linear_regression.create(training, target='price', features=all_features, l2_penalty=0., 
                                          l1_penalty=l1_penalty, validation_set=None, verbose=False)
            for l1_penalty in l1_penalty_values] 

In [26]:
RSSes_3 = [sum((model.predict(validation) - validation['price'])**2) for model in models_3]
NNZs_3 = [model['coefficients']['value'].nnz() for model in models_3]

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [27]:
indexes = [i for i in range(len(NNZs_3)) if NNZs_3[i]==max_nonzeros]
print indexes

[11, 12, 13, 14]


In [31]:
models_3_fixed = [models_3[i] for i in indexes]
RSSes_3_fixed = [RSSes_3[i] for i in indexes]
best_3 = sorted(zip(models_3_fixed, RSSes_3_fixed, indexes), key=lambda x: x[1])[0]

In [32]:
best_3[0]['coefficients'].sort('value', ascending=False)

name,index,value,stderr
(intercept),None,222253.192544,None
bathrooms,None,15873.9572593,None
grade,None,2899.42026975,None
sqft_living_sqrt,None,690.114773313,None
bedrooms,None,661.722717782,None
sqft_living,None,32.4102214513,None
sqft_above,None,30.0115753022,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [33]:
best_3[1]

1046937488751713.4

In [34]:
best_3[2]

11

In [35]:
l1_penalty_values[_]

3448968612.1634369

In [36]:
l1_penalty_values[11]

3448968612.1634369